<a href="https://colab.research.google.com/github/Ying-Yuan07/TensorFlowLearn/blob/main/app%E4%BD%BF%E7%94%A8%E6%8E%A8%E8%8D%90Attention_on_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
%tensorflow_version 1.x
!pip install keras==2.1.6

import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from keras.layers import Input,Dense,LSTM,Activation,RepeatVector,Permute,Flatten,Multiply
from keras.models import Sequential
from keras.models import load_model,Model
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
np.random.seed(123)

#Read data into pandas dataframe
df=pd.read_csv('./sample_data/username_100.csv')
df=df[['week', 'hour_part','app_name']]

#Rename the columns
df.columns=['week', 'hour_part','app_name']
"""
df=df.sort_values(['reviewTime'],ascending=[True])
"""
#Divide the train and test data
#取数据0-1500行为训练集，1500-2000为测试集

length=int(df.shape[0]*0.7)
test_end = 61560
train_data=df[:length]
test_data=df[length:]
print("train_data.shape, test_data.shape")
print(train_data.shape, test_data.shape)
train_data = train_data.reset_index()
test_data = test_data.reset_index()
    
#find item sequences by user in train and test
#按用户id排序，统计每个用户购买商品（item_id）的情况，将同一个用户购买的商品（item_id）放入同一个list，所有list组成一个array
item_seq_train=train_data.groupby("hour_part")['app_name'].apply(list).values
item_seq_test=test_data.groupby("hour_part")['app_name'].apply(list).values
print("item_seq_train.shape")
print(item_seq_train.shape)
print("item_seq_test.shape")
print(item_seq_test.shape)
print("item_seq_test[0]")
print(item_seq_test[0])
# We consider the minimum sequence length as 6. If length is less than 6, we append dummy ids to the sequence  
# 一个用户购买商品数不足6个时，用'padding_id'填充到6个
def min_six_len_seq(dictList):
    new_list=[]
    for i in range(0,len(dictList)):
        if len(dictList[i])<6:
            w=6-len(dictList[i])
            dictList[i]=['app0']*w+dictList[i]
    return dictList


#find the train and test sequences of min len 6
item_train = min_six_len_seq(item_seq_train)
item_test = min_six_len_seq(item_seq_test)
print("item_test[0]")
print(item_test[0])

# train word2vec model on train_data to get item embeddings
#Word2Vec(train_data,size,window,min_count,iter)
#size是指词向量的维度，默认为100
#min_count:要计算词向量的最小词频。这个值可以去掉一些很生僻的低频词，默认是5.可以对字典做截断， 词频少于min_count次数的单词会被丢弃掉
#window:即词向量上下文最大距离,默认为5。window越大，则和某一词较远的词也会产生上下文关系，一般为[5,10]
#iter: 随机梯度下降法中迭代的最大次数，默认是5。对于大语料，可以增大这个值
#workers：用于控制训练的并行数。
def word2vec_model(train_data):
    model = Word2Vec(train_data,size=10,window = 3,min_count =1,iter=5)
    return model


#train the model and save
wv_model=word2vec_model(item_train)
wv_model.save('word2vec_model')

#find full vocabulary
entire_products=[]
i = 0
for key,value in wv_model.wv.vocab.items():
    entire_products.append(key)
np.save('./entire_products.npy',entire_products)
print("len(entire_products)")
print(len(entire_products))
print("entire_products")
print(entire_products)

# Divide the sequences into length of 6. ( First 5 items are for train, 6th one for target)
#用5个历史购买商品预测下一个商品
def input_sequences(new_list, win_size=5):
    input_seq=[]
    target=[]
    for i in range(0,len(new_list)):
        seq_len = len(new_list[i])
        for j in range(0,seq_len):
            if j+win_size<seq_len:
                if new_list[i][j+5] in entire_products:
                    input_seq.append(new_list[i][j:j+5])
                    target.append(new_list[i][j+5])
    return input_seq,target

# Encoding the target.If new item arrives which i
# 将目标商品列表target(原始数据是商品id)转换成商品词汇库中的序号
def num_products(target):
    product_label=LabelEncoder()
    product_label.fit(entire_products)
    target_int = product_label.transform(target)
    return target_int
    
#Create Train and test input and target sequences
#train_x:5个历史购买商品id list组成的arrary,train_y:目标商品在商品词汇库中对应的序号list
train_x,target_train=input_sequences(item_seq_train)
train_y=num_products(target_train)

test_x,target_test=input_sequences(item_seq_test)
test_y=num_products(target_test)
    
#represent each item with prod2vec embedding and if new item comes in test set, represent it with random vec
#将train_x中的每一个商品id都映射成一个维度为10的向量
unknown_item_id=np.random.random((10,))
def w2v_data_extraction(new_list):
    w2v_data=[]
    for i in range(0,len(new_list)):
        seq_vec=[]
        for j in range(0,len(new_list[i])):
            try:
                embedding=wv_model.wv[new_list[i][j]]
            except KeyError:
                embedding=unknown_item_id
            seq_vec.append(embedding)
                
        w2v_data.append(seq_vec)
    return np.asarray(w2v_data)


train_x_emb=w2v_data_extraction(train_x)
test_x_emb=w2v_data_extraction(test_x)
print("train_x_emb.shape")
print(train_x_emb.shape)
#model architecture
def model_arch():
    main_input = Input(shape=(5,10), name='main_input')
    lstm_out = LSTM(32)(main_input)
    attention = Dense(1, activation='tanh')(lstm_out)
    attention = Activation('softmax')(attention)
    attention = RepeatVector(32)(attention)
    attention = Permute([2, 1])(attention)
    attention = Flatten()(attention)
    attention_mul = Multiply()([lstm_out, attention])
    main_output = (Dense(total_vocab, activation='softmax', name='main_output')(attention_mul))
    model = Model(inputs=main_input, outputs=main_output)
    model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='ADAM')
    return model


#represent output as one-hot encoded
def one_hot(seq,total_vocab):
    seq_one_hot=np.zeros([len(seq),total_vocab])
    for i in range(0,len(seq)):
        seq_one_hot[i][seq[i]]=1
    return seq_one_hot

#fit the model on our data
def model_fit(model,train_x,train_y,total_vocab):
    print("train_x.shape")
    print(train_x.shape)
    print("train_y")
    print(train_y.shape)
    train_y=one_hot(train_y,total_vocab)
    print("after one_hot train_y,total_vocab")
    print(train_y.shape)
    print(total_vocab)

    print("model is building")
    model.fit(batch_size=64,epochs=50,x=train_x,y=train_y)
    print("model building done")
    model.save('keras_model.h5')
    
    return model


total_vocab=len(entire_products)
model = model_arch()
model=model_fit(model,train_x_emb,train_y,total_vocab)

# Hit rate at 1 on test data
def hit_rate_at_1(prediction,actual):
    return accuracy_score(prediction,actual)


# 召回率 将正类预测为正类/原本的正类 = TP/(TP+FP)
# Hit rata at 5 on test data
# pred 返回的是一个n行k列的数组，行代表样本，j代表标签
# 第i行第j列上的数值是模型预测第i个预测样本为第j个标签的概率，并且每一行的概率和为1
def hit_rate_at_5(pred,actual):
    predics = []
    for i in range(0, len(pred)):
        predics.append(np.argsort(pred[i])[-5:])#取出第i行，数值top5的下标放入predics，即筛选第i个样本预测概率top5的目标索引
    count = 0
    for i in range(0, len(predics)):#在n个样本预测的结果中，命中的次数
        if actual[i] in predics[i]:
            count = count + 1

    return count/len(actual)


# Hit rate at 10 on test data
def hit_rate_at_10(pred, actual):
    predics = []
    for i in range(0, len(pred)):
        predics.append(np.argsort(pred[i])[-10:])
    count = 0
    print("predics10")
    print(predics)
    for i in range(0, len(predics)):
        if actual[i] in predics[i]:
            count = count + 1
    return count /len(actual)
    
# Prediction on test data
def model_predict(model,test_x,test_seq):

    pred=model.predict(x=test_x)
    preddy=np.argmax(a=pred,axis=1)

    print("pred")
    print(pred)
    print("pred.shape")
    print(pred.shape)
    print("preddy")
    print(preddy)
    print("preddy.shape")
    print(preddy.shape)

    print("hit_rate_at_1(preddy,test_seq)")
    print(hit_rate_at_1(preddy,test_seq))
    print("hit_rate_at_5(preddy,test_seq)")
    print(hit_rate_at_5(pred, test_seq))
    print("hit_rate_at_10(preddy,test_seq)")
    print(hit_rate_at_10(pred, test_seq))

#predict on test data
model_predict(model,test_x_emb,test_y)
print("Done")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
train_data.shape, test_data.shape
(43094, 3) (18470, 3)
item_seq_train.shape
(6,)
item_seq_test.shape
(6,)
item_seq_test[0]
['YouTube', 'YouTube', 'Google Play Store', 'Maps', 'Maps', 'Contacts', 'Contacts', 'Contacts', 'Contacts', 'Maps', 'Maps', 'Maps', 'Google Chrome', 'Samsung Gallery', 'Maps', 'Maps', 'Gmail', 'Maps', 'Maps', 'Maps', 'Contacts', 'Maps', 'Maps', 'Maps', 'Contacts', 'Contacts', 'Maps', 'Maps', 'Google', 'Maps', 'YouTube', 'Maps', 'Contacts', 'Maps', 'Contacts', 'Contacts', 'Contacts', 'Contacts', 'Instagram', 'Contacts', 'Instagram', 'Instagram', 'Instagram', 'Instagram', 'Instagram', 'Google', 'Instagram', 'Instagram', 'Facebook Messenger', 'Gmail', 'Facebook Messenger', 'Gmail', 'Facebook', 'Facebook', 'Facebook Messenger', 'Facebook', 'Facebook Messenger', 'Facebook Messenger', 'Facebook Messenger', 'Facebook Messenger', 'Facebook Messenger', 'Facebook Messenger', '

In [8]:
x = np.array([3, 1, 2,10,7,22,4,500,33,90,77])
print(np.argsort(x))
print(np.argsort(x)[-5:])

[ 1  2  0  6  4  3  5  8 10  9  7]
[ 5  8 10  9  7]
